In [ ]:
from AWSIoTPythonSDK.MQTTLib import AWSIoTMQTTShadowClient
import numpy as np
import random, time
import pandas as pd
import json
import time
import datetime

In [ ]:
# A random programmatic shadow client ID.
SHADOW_CLIENT = "A0169326X_R2R_ShadowClient"

# The unique hostname that &IoT; generated for this device.
HOST_NAME = "a1ck22vikbcyjq-ats.iot.us-west-2.amazonaws.com"

# The relative path to the correct root CA file for &IoT;, which you have already saved onto this device.
ROOT_CA = "F:/19201/EE5111/AWS/CA/R2R/AmazonRootCA1.pem"

# The relative path to your private key file that &IoT; generated for this device, which you have already saved onto this device.
PRIVATE_KEY = "F:/19201/EE5111/AWS/CA/R2R/026d0604b4-private.pem.key"

# The relative path to your certificate file that &IoT; generated for this device, which you have already saved onto this device.
CERT_FILE = "F:/19201/EE5111/AWS/CA/R2R/026d0604b4-certificate.pem.crt.txt"

# A programmatic shadow handler name prefix.
SHADOW_HANDLER = "A0169326X_R2R"


In [ ]:
# Automatically called whenever the shadow is updated.
def myShadowUpdateCallback(payload, responseStatus, token):
    print()
    print('UPDATE: $aws/things/' + SHADOW_HANDLER + '/shadow/update/#')
    print("payload = " + payload)
    print("responseStatus = " + responseStatus)
    print("token = " + token)

# Create, configure, and connect a shadow client.
print('Connecting to AWS IoT...')
myShadowClient = AWSIoTMQTTShadowClient(SHADOW_CLIENT)
myShadowClient.configureEndpoint(HOST_NAME, 8883)
myShadowClient.configureCredentials(ROOT_CA, PRIVATE_KEY, CERT_FILE)
myShadowClient.configureConnectDisconnectTimeout(10)
myShadowClient.configureMQTTOperationTimeout(5)
myShadowClient.connect()

In [ ]:
# Create a programmatic representation of the shadow.
myDeviceShadow = myShadowClient.createShadowHandlerWithName(SHADOW_HANDLER, True)

# Keep sending data from FD002.txt until this script stops running.
# To stop running this script, press Ctrl+C.
df_T2 = pd.read_excel('F:/19201/EE5111/AWS/CA/R2R/Tem100_v1to6_Ten2.xlsx', delim_whitespace=True, header=None)
size_df = df_T2.size
num_rows = size_df//28
sensor_name = ['s'+ str(i) for i in range(1,len(df_T2.columns))]
df_T2.columns = ['id'] + sensor_name
df_T2['id'] = df_T2['id'].apply(lambda idnum: 'T2_' + str(idnum))
df_T2['Matriculation NO.'] = 'A0169326X'
df_T2['timestamp'] = str(np.nan)

for i in range(10,num_rows):
    now_stamp = time.time()
    local_time = datetime.datetime.fromtimestamp(now_stamp)
    utc_time = datetime.datetime.utcfromtimestamp(now_stamp)
    df_T2.at[i,'timestamp'] = str('UTC'+' '+str(utc_time))
    json_format = df_T2.iloc[i].to_json()
    display_form_json = '{"state":{"reported":'+json_format+'}}'
    
    try:
        myDeviceShadow.shadowUpdate(display_form_json,myShadowUpdateCallback, 5)
    except:
        print('The' + str(i) + '-th row data cannot be displayed!')
        continue
    time.sleep(0.05)